# PyMem3DG Tutorial 5 - Extended Applications 1: membrane heterogeneity
`Cuncheng Zhu, Christopher T. Lee`

This tutorial covers how `Mem3DG` prescribes membrane heterogeneity to model local membrane effect. In previous tutorials, we only simulate membrane with homogenous composition and therefore constant membrane properties. However, in reality, most biological processes require local concentration of biological cues to drive more complex membrane response. For example, it has been suggested that local binding of Clathrin scaffolds the membrane to induce endocytosis. 

To demonstrate, as in tutorial 3, we will set up single boundary patch simulation. Please review if neccesary. In tutorial 3, where the deformation is mainly driven by the homogenous osmotic pressure. 

## Bending-driven budding
This tutorial will model local scaffolding effect of Clathrin assuming the scaffold induced nonzero spontaneous curvature and the scaffold is more rigid than the bare membrane.

In [1]:
import pymem3dg as dg

hexFace, hexVertex = dg.getHexagon(R = 1, nSub = 4)

o = dg.Options()
p = dg.Parameters()

o.isConstantSurfaceTension = True
o.isConstantOsmoticPressure = True
o.shapeBoundaryCondition = "fixed"
o.isEdgeFlip = True
o.isSplitEdge = True
o.isCollapseEdge = True
o.isVertexShift = True

# surface tension
p.Ksg = 1e-3

# osmotic pressure
p.Kv = 0
p.Vt = -1
p.V_res = 0
p.cam = -1

We have already seen the settings above in tutorial 3. To reiterate, we instantiate a hexagon patch mesh, enable all mesh mutation operation to ensure mesh quality, and apply a constant surface tension and zero osmotic pressure. 

Unlike the previous simulation, in this tutorial we want to prescribe a heterogeneity patch. The parametrization is done using the protein density $\phi \in [0,1]$. `protein0` is used to specify the constant protein distribution or initial protein distribution when we talk about protein dynamics in the next tutorial. There are several ways to initialize it. First, when given single scalar $\phi^0$, it prescribes uniform distribution of $\phi = \phi^0$; second when given full vector ${\phi^0_i}$ with the number of entries equals to the number of vertices, it prescribes elementwise distribution in the order of vertex indices; the thrid method, which is applied in the following example, is to utilize the geodesic distance computation. 

In this case, we will initialize `protein0` *= \[$R_1$, $R_2$, $\phi^{in}$, $\phi^{out}$\]* drawing geodesic ellipse with two principal radius $R_1$ and $R_2$ centered at one particular point `pt` of the surface, and then precribing protein density inside $\phi^{in}$ and outside $\phi^{out}$, as well the sharpness of transition `sharpness` across the phase separation. 

To specify `pt`, you could either specify in 2D or 3D such that it will find the cloest point on the suface to the embedded point in term of Euclidean distance. `isFloatVertex` option repetitively compute the surface point to find parametrized point on the surface rather than on particular mesh vertex if turned off. Please refer to the documentation for more implementational details 

In [2]:
o.isFloatVertex = True
p.pt = [0, 0] # 
p.protein0 = [0.5, 0.5, 1, 0]
p.sharpness = 20

# bending
p.Kb = 8.22e-5
p.Kbc = 3 * 8.22e-5 
p.H0c = 6

As protein density will affect the membrane property, we need to specify the constituitive relation. Currently, *Mem3DG* supports the heterogeneous bending rigidity and spontaneous curvature to model endocytic patch. The vertex-wise bending rigidity $\kappa = K_b + K_{bc} ~\phi$ with user-defined parameter `Kb` and `Kbc`. Similarly, the vertex-wise spontaneous curvature $H_0 = H_{0c} ~\phi$ with user-defined `H0c`. In other words, in this particular setup, we coated the center patch with protein with preferred curvature of 6 $\mu m^{-1}$ and the coated membrane with scaffolding is 4 times more rigid than the bare membrane.

In [3]:
g = dg.System(hexFace, hexVertex, p, o)

g.meshMutator.flipNonDelaunay = True
g.meshMutator.splitLarge = True
g.meshMutator.splitFat = True
g.meshMutator.splitSkinnyDelaunay = True
g.meshMutator.splitCurved = True
g.meshMutator.curvTol = 0.005
g.meshMutator.collapseSkinny = True

The initial configuration with the colormap showing the protein density is the following:

<img src="output/tutorial5/phi.png" width="450" height="225">

where $\phi = 1$ within the circular patch of radius 0.5 centered at \[0, 0\] and $\phi = 0$ outside the patch.

With `System` **g** initilized, the rest is just routinely carrying out the integrator such as the forward Euler method. One particular things to note is that we could specify the period of recomputing geodesics on surface mesh, as well as that of doing mesh processing to ensure mesh quality using integrator options `tUpdateGeodesics` and `tProcessMesh`, respectively. 

In [4]:
fe = dg.Euler(f = g, dt = 0.05, total_time = 20000, tSave = 100, tolerance = 3e-5, outputDir = "output/tutorial5/Kb/traj")
fe.tUpdateGeodesics = 50
fe.tProcessMesh = 50
# fe.integrate()

The resultant budding trajectory snapshots at `T = 0, 10000, 15100, 18100 and 20000` with the colormap showing the bending force:

<img src="output/tutorial5/Kb/screenshot_frame000000.png" width="400" height="200">
<img src="output/tutorial5/Kb/screenshot_frame000100.png" width="400" height="200">
<img src="output/tutorial5/Kb/screenshot_frame000151.png" width="400" height="200">
<img src="output/tutorial5/Kb/screenshot_frame000181.png" width="400" height="200">
<img src="output/tutorial5/Kb/screenshot_frame000200.png" width="400" height="200">

and a transparent visualization of of the mean curvature for `T = 20000`:

<img src="output/tutorial5/Kb/H_final.png" width="400" height="200">

## Interfacial line tension driven budding 
Last example we computationally model the scaffolding mechanism where we assuming the coated membrane is significantly more rigid than bare membrane. In the following section, we could provides an alternative budding mechanism by introducing additional energy terms. The following study should also be treated as an blueprint for `Mem3DG` to grow and further accomodate more complex physics as we develop. Please refer to the detailed documentation or relevant publication for theorical and implemental explantions. 

The additional is the Dirichlet energy $E_d = \frac{1}{2} \int_{\mathcal{M}} \eta \| \nabla_{\vec{\theta}} \phi \|^2 dA$ with user-defined parameter `eta` to control the role of the energy to the system. Intuitively the energy penalize any nonsmoothness of the protein distribution. Mechanically in our case with assumed phase seperation, it will results in the so-called "line tension" that constrict the interface to form the neck area of membrane budding. Chemically when we consider protein dynamics later in the next tutorial, it will result in protein diffusion.  

In [5]:
p.eta = 5e-4
p.Kbc = 8.22e-5
h = dg.System(hexFace, hexVertex, p, o)
h.meshMutator.flipNonDelaunay = True
h.meshMutator.splitLarge = True
h.meshMutator.splitFat = True
h.meshMutator.splitSkinnyDelaunay = True
h.meshMutator.splitCurved = True
h.meshMutator.curvTol = 0.005
h.meshMutator.collapseSkinny = True

fe = dg.Euler(f = h, dt = 0.1, total_time = 20000, tSave = 100, tolerance = 3e-5, outputDir = "output/tutorial5/eta")
fe.tUpdateGeodesics = 50
fe.tProcessMesh = 50
# fe.integrate()
 


The resultant budding trajectory snapshots at `T = 0, 2000, 5000, 6000 and 7300` with the colormap showing the line tension force:

<img src="output/tutorial5/eta/screenshot_frame000000.png" width="400" height="200">
<img src="output/tutorial5/eta/screenshot_frame000020.png" width="400" height="200">
<img src="output/tutorial5/eta/screenshot_frame000050.png" width="400" height="200">
<img src="output/tutorial5/eta/screenshot_frame000060.png" width="400" height="200">
<img src="output/tutorial5/eta/screenshot_frame000073.png" width="400" height="200">

and a transparent visualization of of the mean curvature for `T = 7300`:

<img src="output/tutorial5/eta/H_73.png" width="400" height="200">